In [2]:
# importing data and auxiliar functions
import sys
sys.path.insert(0, '..')

import saopaulo.sp_grid as gr
import saopaulo.cycling_potential as cp
from bikescience.slope import plot_slope, plot_slopes, split_route
from shapely.geometry import LineString
import ast

from math import floor

import folium
import pandas as pd
import geopandas as gpd
from statistics import stdev
from ipywidgets import interact_manual, widgets, fixed
from IPython.core.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import warnings
import requests
import os
warnings.simplefilter('ignore')
import time
from operator import itemgetter

from pyproj import Transformer
import datetime


/home/pedro/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
def string_to_linestring(string):
    if string[0] == 'L': # Linestring format
        content = ''
        for point in string[14:-1].split(','):
            content += '['
            for p in point.split(' '):
                if p == '': continue
                content += str(p)
                content += ','
            content = content[0:-1]
            content += '],'
        content = content[0:-1]
        string = '[' + content + ']'
        
    return LineString(eval(string))

In [5]:
complete_csv = pd.read_csv('../bases/complete_csv.csv')
complete_csv = complete_csv[complete_csv['FE_VIA'] > 0]
complete_csv['ID_ORDEM'] = complete_csv['id']
complete_csv.index = complete_csv['ID_ORDEM']

# fix FE_VIA with wrong format (because of thousand separator)
complete_csv.loc[complete_csv['FE_VIA'] >= 10000, 'FE_VIA'] = complete_csv.loc[complete_csv['FE_VIA'] >= 10000, 'FE_VIA'] / 1000
complete_csv.set_index('ID_ORDEM', inplace=True, drop=False)
complete_csv = gpd.GeoDataFrame(complete_csv, crs={'init': 'epsg:4326'},
                 geometry=[string_to_linestring(r) for r in complete_csv['route']])

In [6]:
# read routes files and join with general trips file
od_trips = pd.read_csv('../../data/sao-paulo/od/OD_2017.csv')
od_trips.columns = [c[: c.index(',')] for c in od_trips.columns]
od_trips.set_index('ID_ORDEM', inplace=True, drop=False)

In [4]:
#display(complete_csv)
#display(od_trips)

In [5]:
df = od_trips.join(other = complete_csv, how = 'left', rsuffix='_complete')
display(df)

,ZONA,MUNI_DOM,CO_DOM_X,CO_DOM_Y,ID_DOM,F_DOM,FE_DOM,DOM,CD_ENTRE,DATA,...,TP_ESTBICI_complete,ID_ORDEM.1,length,modal,id,distance_potential,age_potential,inclination_potential,final_potential,final_potential_v2
ID_ORDEM,,,,,,,,,,,,,,,,,,,,,
1,1,36,333743,7394463,10001,1,15.416667,1,1,6092017,...,0.0,1.0,1849.286293,pedestrian,1.0,0.650491,0.023107,0.206501,0.293366,0.428496
2,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,0.0,2.0,1760.427711,pedestrian,2.0,0.689523,0.023107,0.463706,0.392112,0.576615
3,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,0.0,3.0,9473.535456,subway,3.0,0.032389,0.848482,0.066909,0.315927,0.049649
4,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,0.0,4.0,6428.546731,subway,4.0,0.077287,0.848482,0.402769,0.442846,0.240028
4,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,0.0,4.0,2667.571706,bus_sp,4.0,0.391186,0.848482,0.280026,0.506565,0.335606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183088,517,25,294618,7410518,5171418,0,85.531250,1418,1,15092018,...,0.0,183088.0,131.295991,pedestrian,183088.0,0.128145,0.323018,0.843343,0.431502,0.485744
183089,517,25,295243,7411456,5171599,1,52.500000,1599,1,15092018,...,0.0,183089.0,816.655729,pedestrian,183089.0,0.905531,0.720850,0.638682,0.755021,0.772107
183090,517,25,295243,7411456,5171599,0,52.500000,1599,1,15092018,...,0.0,183090.0,752.109085,pedestrian,183090.0,0.826529,0.720850,0.441549,0.662976,0.634039


In [6]:
display(df.loc[(df['ID_ORDEM_complete'].isnull()) & 
               (df['FE_VIA'] > 0) &
               (df['MODO1'] != 15)])

,ZONA,MUNI_DOM,CO_DOM_X,CO_DOM_Y,ID_DOM,F_DOM,FE_DOM,DOM,CD_ENTRE,DATA,...,TP_ESTBICI_complete,ID_ORDEM.1,length,modal,id,distance_potential,age_potential,inclination_potential,final_potential,final_potential_v2
ID_ORDEM,,,,,,,,,,,,,,,,,,,,,


## Processing records with errors

In [7]:
graph_hopper_url = 'https://graphhopper.com/api/1/route'
graph_hopper_keys = ['bb4601e7-696d-4b91-a809-4b1b8256b2a8', # yahoo higor
                     'd5609235-850b-44c1-a260-2913dc9a89d5', # hotmail pedro
                     '4e761e67-50ce-40ad-bda3-df7cce5567eb', # gmail higor
                     'af8321e8-0a13-4d77-b9e6-2a6b01f6feab'  # gmail pedro
                    ] 


def get_route_from_coordinates (origin, destination, key = 0):
    url =  graph_hopper_url + '?key=' + graph_hopper_keys[key] + '&point={},{}&point={},{}&' + \
           'vehicle=bike&elevation=true&type=json&points_encoded=false'

    result = requests.get(url.format(origin[0], origin[1],destination[0],destination[1]))
    if result.status_code == 200:
        path = result.json()['paths'][0]
        route = path['points']['coordinates']
        
        geo = LineString(route)
        return geo
    else:
        print(result.status_code)
        return None
    
def convert_projected_to_latlon(x, y):
    # 31983
    # 32723
    transformer = Transformer.from_crs("EPSG:31983", "EPSG:4326")

    return transformer.transform(x, y)

In [8]:
def get_coords_transfers(row):
    x_y = [[row['CO_O_X'], row['CO_O_Y']]]
    if row['CO_T1_X'] > 0:
        x_y.append([row['CO_T1_X'], row['CO_T1_Y']])
    if row['CO_T2_X'] > 0:
        x_y.append([row['CO_T2_X'], row['CO_T2_Y']])
    if row['CO_T3_X'] > 0:
        x_y.append([row['CO_T3_X'], row['CO_T3_Y']])
    
    x_y.append([row['CO_D_X'], row['CO_D_Y']])

    coords = []
    for c in x_y:
        coords.append(convert_projected_to_latlon(c[0], c[1]))
        
    return coords

def get_routes(od_record):
    coords = get_coords_transfers(od_record)
    if (od_record['MODO1'] == 0): return []
    
    data = []
    for i in range(1, len(coords)):
        o = coords[i - 1]
        d = coords[i]
        r = get_route_from_coordinates(o, d)
        
        data.append([od_record['ID_ORDEM'], od_record['MODO' + str(i)], i, o[0], o[1], d[0], d[1], r])
        
    return data

In [9]:
print(complete_csv['modal'].unique())

['pedestrian' 'subway' 'bus_sp' 'taxi_app' 'car' 'bus_private' 'train'
 'taxi' 'bus_school' 'other' 'motorcycle' 'bus_inter' 'bus_otherc'
 'monorail' '0']


In [10]:
modal_od = [
    0,
    "subway",    
    "train",
    "monorail",
    "bus_sp",
    "bus_otherc",
    "bus_inter",
    "bus_private",
    "bus_school",
    "car",
    "car",
    "taxi",
    "taxi_app",
    "motorcycle",
    "motorcycle",
    "bicycle",
    "pedestrian",
    "other"
]

In [11]:
id_ordem = 450
display(od_trips.loc[od_trips['ID_ORDEM'] == id_ordem][['ID_ORDEM', 'FE_VIA', 'MODOPRIN', 'MODO1', 'MODO2', 'MODO3', 'MODO4']])
display(complete_csv.loc[complete_csv['ID_ORDEM'] == id_ordem][['ID_ORDEM', 'FE_VIA', 'modal']])

,ID_ORDEM,FE_VIA,MODOPRIN,MODO1,MODO2,MODO3,MODO4
ID_ORDEM,,,,,,,
450,450,45.993065,15,15,0,0,0


,ID_ORDEM,FE_VIA,modal
ID_ORDEM,,,


In [12]:
def find_errors():
    errors_aux = []
    trips = 0
    last_id = 0

    for _, r in od_trips.iterrows():
    #for e in errors6:
        trips += 1
        if trips % 100 == 0: 
            print (datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), trips, len(od_trips), len(errors6))
        
        id_ordem = r['ID_ORDEM']
        #id_ordem = e[1]
        if id_ordem == last_id:
            continue
        last_id = id_ordem
        #r = od_trips.loc[od_trips['ID_ORDEM'] == id_ordem].iloc[0]
        
        coords = get_coords_transfers(r)
        aux = complete_csv.loc[(complete_csv['ID_ORDEM'] == r['ID_ORDEM'])]

        for i in range(len(coords) - 1):
            modal = modal_od[int(r['MODO' + str(i + 1)])]
            if modal == 'bicycle':
                continue

            csv_row = aux.loc[(aux['modal'] == modal) & 
                              (round(aux['orig_lat'], 5) == round(coords[i][0], 5))]

            if len(csv_row) == 0 and r['FE_VIA'] > 0:
                errors_aux.append(['Error not found', r['ID_ORDEM'], i + 1, modal])

        if len(aux) > len(coords) - 1:
            errors_aux.append(['Error more trips than it should', r['ID_ORDEM'], len(coords) - 1, len(aux)])

    return errors_aux

In [14]:
#errors8 = find_errors()
#file = open('errors8.txt','w')
#file.writelines(str(errors8))
#print(len(errors8))
#file.close()

In [15]:
#with open('errors.txt') as f:
#    errors_str = f.read()
#    errors = ast.literal_eval(errors_str)
#    print(len(errors))
#if errors == []:

In [ ]:
#error_df = pd.DataFrame(errors4, columns = ['Description', 'ID ORDEM', 'info1', 'info2'])
#display(error_df)

In [ ]:
#error_df.loc[error_df['Description'] == 'Error not found'].groupby('info2')['info2'].count()

In [16]:
last_index = 920000

In [17]:
def reprocess(errors):
    # regenerating the routes of trips with errors
    new_errors = []
    trips = 0
    last_id = 0
    key = 3

    for error in errors:
        if trips % 50 == 0: 
            print (datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), trips, len(errors7), len(new_errors))
        trips += 1

        id_ordem = error[1]
        #if id_ordem < 51233: continue
        if id_ordem == last_id: continue

        try:
            complete_csv.drop(complete_csv.loc[complete_csv['ID_ORDEM'] == id_ordem].index, inplace=True)
        except:
            print(id_ordem, 'não encontrado')


        od_record = od_trips.loc[od_trips['ID_ORDEM'] == id_ordem].iloc[0]

        coords = get_coords_transfers(od_record)

        for i in range(1, len(coords)):
            modal = modal_od[int(od_record['MODO' + str(i)])]
            if modal == 'bicycle':
                continue

            o = coords[i - 1]
            d = coords[i]
            r = get_route_from_coordinates(o, d, key)

            if r == None:
                new_errors.append([id_ordem, i])
                print(id_ordem, i, 'rota não gerada')
                if key > 0:
                    key -= 1
            else:
                od_record['orig_lat'] = o[0]
                od_record['orig_lon'] = o[1]
                od_record['dest_lat'] = d[0]
                od_record['dest_lon'] = d[1]
                od_record['route'] = r
                od_record['length'] = r.length
                od_record['modal'] = modal
                od_record['id'] = id_ordem
                od_record['distance_potential'] = 0
                od_record['age_potential'] = 0
                od_record['inclination_potential'] = 0
                od_record['final_potential'] = 0
                od_record['final_potential_v2'] = 0

                last_index += 1
                complete_csv.loc[last_index] = od_record

        #display(complete_csv.loc[complete_csv['id'] == id_ordem])
        last_id = id_ordem
        if len(new_errors) > 10:
            print(trips, last_index, last_id)
            break

In [35]:
sum(complete_csv['FE_VIA'])

49273751.4371459

In [88]:
display(sum(complete_csv.loc[complete_csv['final_potential'] == 0]['FE_VIA']))

13454.036378000003

In [83]:
c = complete_csv.columns
i_v1 = c.get_loc('final_potential')
i_v2 = c.get_loc('final_potential_v2')
i_dist = c.get_loc('distance_potential')
i_age = c.get_loc('age_potential')
i_incl = c.get_loc('inclination_potential')
print(i_v1, i_v2, i_dist, i_age, i_incl)

139 140 136 137 138


In [84]:
errors_potential = []
potentials = []
n = len(complete_csv)
for i in range(len(complete_csv)):
    if i % 1000 == 0: 
        print (datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), i, n, round(i / n * 100, 2))
    i += 1
    
    try:
        pot_d, pot_a, pot_i = cp.cycling_potential_variables(complete_csv.iloc[i])
        v1 = (pot_d + pot_a + pot_i) / 3
        v2 = (pot_d + pot_i) / 2
        
        complete_csv.iat[i, i_dist] = pot_d
        complete_csv.iat[i, i_age] = pot_a
        complete_csv.iat[i, i_incl] = pot_i
        complete_csv.iat[i, i_v1] = v1
        complete_csv.iat[i, i_v2] = v2
        
        if v1 == 0: print('null potential ? ', i)
    except Exception as ex:
        print(i, ex)
        pot_d, pot_a, pot_i, v1, v2 = 0,0,0,0,0
        errors_potential.append(i)
        # broken cases usually represent inconsistent routes
    potentials.append([i, pot_d, pot_a, pot_i, (pot_d + pot_a + pot_i) / 3])
        

2023-05-16 21:27:14 0 180238 0.0
2023-05-16 21:27:25 1000 180238 0.55
2023-05-16 21:27:35 2000 180238 1.11
2023-05-16 21:27:46 3000 180238 1.66
3062 float division by zero
3063 float division by zero
2023-05-16 21:27:55 4000 180238 2.22
4164 float division by zero
4165 float division by zero
4168 float division by zero
4169 float division by zero
2023-05-16 21:28:06 5000 180238 2.77
5871 float division by zero
5872 float division by zero
2023-05-16 21:28:17 6000 180238 3.33
2023-05-16 21:28:29 7000 180238 3.88
2023-05-16 21:28:40 8000 180238 4.44
2023-05-16 21:28:49 9000 180238 4.99
2023-05-16 21:29:00 10000 180238 5.55
2023-05-16 21:29:11 11000 180238 6.1
2023-05-16 21:29:21 12000 180238 6.66
2023-05-16 21:29:32 13000 180238 7.21
2023-05-16 21:29:42 14000 180238 7.77
2023-05-16 21:29:53 15000 180238 8.32
2023-05-16 21:30:04 16000 180238 8.88
2023-05-16 21:30:16 17000 180238 9.43
2023-05-16 21:30:27 18000 180238 9.99
2023-05-16 21:30:38 19000 180238 10.54
2023-05-16 21:30:49 20000 1802

170394 float division by zero
2023-05-16 22:03:16 171000 180238 94.87
171343 float division by zero
2023-05-16 22:03:28 172000 180238 95.43
172208 float division by zero
172243 float division by zero
172684 float division by zero
172814 float division by zero
172869 float division by zero
172870 float division by zero
2023-05-16 22:03:41 173000 180238 95.98
173793 float division by zero
2023-05-16 22:03:54 174000 180238 96.54
174502 float division by zero
174504 float division by zero
174564 float division by zero
2023-05-16 22:04:08 175000 180238 97.09
175183 float division by zero
2023-05-16 22:04:24 176000 180238 97.65
2023-05-16 22:04:39 177000 180238 98.2
177805 float division by zero
2023-05-16 22:04:52 178000 180238 98.76
178226 float division by zero
178236 float division by zero
178350 float division by zero
178351 float division by zero
178750 float division by zero
178751 float division by zero
178857 float division by zero
2023-05-16 22:05:08 179000 180238 99.31
179128 floa

In [86]:
print(len(errors_potential))
print(len(errors_potential) / len(complete_csv))

75
0.0004161164682253465


In [89]:
complete_csv.to_csv('complete_csv.csv')

In [7]:
lite = complete_csv.reset_index(drop=True)
display(lite)

,ID_ORDEM,ID_ORDEM.1,ID_ORDEM.1.1,orig_lat,orig_lon,dest_lat,dest_lon,route,ZONA,MUNI_DOM,...,ID_ORDEM.1.1.1,length,modal,id,distance_potential,age_potential,inclination_potential,final_potential,final_potential_v2,geometry
0,1.0,1.0,1.0,-23.551678,-46.628858,-23.551495,-46.635115,"[[-46.628827, -23.55186, 748.52], [-46.627845,...",1.0,36.0,...,1.0,1849.286293,pedestrian,1.0,0.650491,0.023107,0.206501,0.293366,0.428496,"LINESTRING Z (-46.62883 -23.55186 748.52000, -..."
1,2.0,2.0,2.0,-23.551495,-46.635115,-23.551678,-46.628858,"[[-46.635147, -23.551546, 783.14], [-46.635381...",1.0,36.0,...,2.0,1760.427711,pedestrian,2.0,0.689523,0.023107,0.463706,0.392112,0.576615,"LINESTRING Z (-46.63515 -23.55155 783.14000, -..."
2,3.0,3.0,3.0,-23.551678,-46.628858,-23.571829,-46.690238,"[[-46.628827, -23.55186, 748.52], [-46.627845,...",1.0,36.0,...,3.0,9473.535456,subway,3.0,0.032389,0.848482,0.066909,0.315927,0.049649,"LINESTRING Z (-46.62883 -23.55186 748.52000, -..."
3,4.0,4.0,4.0,-23.571829,-46.690238,-23.525949,-46.666559,"[[-46.690264, -23.571842, 742.26], [-46.691246...",1.0,36.0,...,4.0,6428.546731,subway,4.0,0.077287,0.848482,0.402769,0.442846,0.240028,"LINESTRING Z (-46.69026 -23.57184 742.26000, -..."
4,4.0,4.0,4.0,-23.525949,-46.666559,-23.537903,-46.670921,"[[-46.666835, -23.525929, 733.11], [-46.666934...",1.0,36.0,...,4.0,2667.571706,bus_sp,4.0,0.391186,0.848482,0.280026,0.506565,0.335606,"LINESTRING Z (-46.66683 -23.52593 733.11000, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180233,55728.0,55728.0,55728.0,-23.506304,-46.687070,-23.546946,-46.629749,"LINESTRING Z (-46.68727 -23.506181 727.59, -46...",138.0,36.0,...,55728.0,0.100071,bus_sp,55728.0,0.056332,0.641112,0.830032,0.509159,0.443182,"LINESTRING Z (-46.68727 -23.50618 727.59000, -..."
180234,55728.0,55728.0,55728.0,-23.546946,-46.629749,-23.602704,-46.602333,LINESTRING Z (-46.629825 -23.54699 732.1799999...,138.0,36.0,...,55728.0,0.074911,bus_sp,55728.0,0.056322,0.641112,0.429314,0.375582,0.242818,"LINESTRING Z (-46.62982 -23.54699 732.18000, -..."
180235,55728.0,55728.0,55728.0,-23.602704,-46.602333,-23.600440,-46.587185,LINESTRING Z (-46.602275 -23.602806 749.429999...,138.0,36.0,...,55728.0,0.022320,bus_sp,55728.0,0.056300,0.641112,0.925131,0.540847,0.490715,"LINESTRING Z (-46.60227 -23.60281 749.43000, -..."
180236,83593.0,83593.0,83593.0,-23.603440,-46.460983,-23.598433,-46.462309,"LINESTRING Z (-46.461086 -23.603493 801.77, -4...",231.0,36.0,...,83593.0,0.008205,bus_sp,83593.0,0.039232,0.817337,0.993869,0.616813,0.516550,"LINESTRING Z (-46.46109 -23.60349 801.77000, -..."


In [11]:
lite.to_file('../bases/routes_potential.shp', driver='ESRI Shapefile')

In [9]:
lite = lite.drop('route', axis=1)


In [10]:
display(lite)

,ID_ORDEM,ID_ORDEM.1,ID_ORDEM.1.1,orig_lat,orig_lon,dest_lat,dest_lon,ZONA,MUNI_DOM,CO_DOM_X,...,ID_ORDEM.1.1.1,length,modal,id,distance_potential,age_potential,inclination_potential,final_potential,final_potential_v2,geometry
0,1.0,1.0,1.0,-23.551678,-46.628858,-23.551495,-46.635115,1.0,36.0,333743.0,...,1.0,1849.286293,pedestrian,1.0,0.650491,0.023107,0.206501,0.293366,0.428496,"LINESTRING Z (-46.62883 -23.55186 748.52000, -..."
1,2.0,2.0,2.0,-23.551495,-46.635115,-23.551678,-46.628858,1.0,36.0,333743.0,...,2.0,1760.427711,pedestrian,2.0,0.689523,0.023107,0.463706,0.392112,0.576615,"LINESTRING Z (-46.63515 -23.55155 783.14000, -..."
2,3.0,3.0,3.0,-23.551678,-46.628858,-23.571829,-46.690238,1.0,36.0,333743.0,...,3.0,9473.535456,subway,3.0,0.032389,0.848482,0.066909,0.315927,0.049649,"LINESTRING Z (-46.62883 -23.55186 748.52000, -..."
3,4.0,4.0,4.0,-23.571829,-46.690238,-23.525949,-46.666559,1.0,36.0,333743.0,...,4.0,6428.546731,subway,4.0,0.077287,0.848482,0.402769,0.442846,0.240028,"LINESTRING Z (-46.69026 -23.57184 742.26000, -..."
4,4.0,4.0,4.0,-23.525949,-46.666559,-23.537903,-46.670921,1.0,36.0,333743.0,...,4.0,2667.571706,bus_sp,4.0,0.391186,0.848482,0.280026,0.506565,0.335606,"LINESTRING Z (-46.66683 -23.52593 733.11000, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180233,55728.0,55728.0,55728.0,-23.506304,-46.687070,-23.546946,-46.629749,138.0,36.0,327741.0,...,55728.0,0.100071,bus_sp,55728.0,0.056332,0.641112,0.830032,0.509159,0.443182,"LINESTRING Z (-46.68727 -23.50618 727.59000, -..."
180234,55728.0,55728.0,55728.0,-23.546946,-46.629749,-23.602704,-46.602333,138.0,36.0,327741.0,...,55728.0,0.074911,bus_sp,55728.0,0.056322,0.641112,0.429314,0.375582,0.242818,"LINESTRING Z (-46.62982 -23.54699 732.18000, -..."
180235,55728.0,55728.0,55728.0,-23.602704,-46.602333,-23.600440,-46.587185,138.0,36.0,327741.0,...,55728.0,0.022320,bus_sp,55728.0,0.056300,0.641112,0.925131,0.540847,0.490715,"LINESTRING Z (-46.60227 -23.60281 749.43000, -..."
180236,83593.0,83593.0,83593.0,-23.603440,-46.460983,-23.598433,-46.462309,231.0,36.0,355887.0,...,83593.0,0.008205,bus_sp,83593.0,0.039232,0.817337,0.993869,0.616813,0.516550,"LINESTRING Z (-46.46109 -23.60349 801.77000, -..."
